In [ ]:
# append path info to access api
import os, sys
#from git_root import git_root
#sys.path.append(os.path.abspath(git_root("nmdc_runtime/api/core")))
#sys.path.append(os.path.abspath(git_root("nmdc_runtime/site")))

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import json
import copy
import pandas as pd
from toolz.dicttoolz import assoc_in, merge
from pprint import pprint
from collections import namedtuple

# from resources import MongoDB
from pymongo import MongoClient
from nmdc_runtime.api.core.metadata import load_changesheet, update_mongo_db
from linkml_runtime.utils.schemaview import SchemaView

### load mongo via env

In [ ]:
from dotenv import load_dotenv

load_dotenv("../../.env.localhost")

In [ ]:
import warnings

import dagster

warnings.filterwarnings("ignore", category=dagster.ExperimentalWarning)

In [ ]:
from nmdc_runtime.site.repository import run_config_frozen__normal_env
from nmdc_runtime.site.resources import get_mongo


mongo_via_env = get_mongo(run_config_frozen__normal_env)

### load change sheet

In [ ]:
pd.set_option("display.max_columns", None)
pd.set_option('display.width', 1000)

In [ ]:
pd.read_csv("data/changesheet-without-separator3.tsv", sep="\t")

In [ ]:
sheetDf = load_changesheet('data/changesheet-without-separator3.tsv')
sheetDf

### ensure indexes on mongodb collections

In [ ]:
from nmdc_runtime.util import nmdc_jsonschema, nmdc_jsonschema_validate

collection_names = {
    name for name, spec in nmdc_jsonschema["properties"].items()
    if name.endswith("_set")
}

In [ ]:
df_change = sheetDf


client = MongoClient(
    host="localhost",
    port=27018,
    username="admin",
    password="root",
)
mongodb = client["nmdc_etl_staging"]

for name in collection_names:
    mongodb[name].create_index("id", unique=True)

def make_id_to_collection_dict(mongodb) -> dict:
    collection_names = [
        name for name in mongodb.list_collection_names() if name.endswith("_set")
    ]
    id_dict = {
        name: set(mongodb[name].distinct("id"))
        for name in collection_names
        if "id_1" in mongodb[name].index_information()
    }
    return id_dict

# create a dict between collections names and ids
id_dict = make_id_to_collection_dict(mongodb)

id_group = df_change.groupby("group_id")

def get_collection_for_id(id_val, id_map):
    for collection_name in id_map:
        if id_val in id_map[collection_name]:
            return collection_name
    return None

updates = []  # list of dicts
for ig in id_group:
    # ig[0] -> id of the data
    # ig[1] -> dataframe with rows with the id

    #         schema = try_fetching_schema_for_id(ig[0])
    collection_name = get_collection_for_id(ig[0], id_dict)

In [ ]:
from toolz import dissoc

study_docs = [dissoc(d, "_id") for d in mongo_via_env.db.study_set.find()]

_ = nmdc_jsonschema_validate({"study_set": study_docs})

mongodb.study_set.drop()
mongodb.study_set.insert_many(study_docs)

for name in collection_names:
    mongodb[name].create_index("id", unique=True)

### update mongodb using change sheet

In [ ]:
update_mongo_db(sheetDf, print_update=True, print_data=True, print_query=True)